# Italian to Greek
* In this Jupyter Notebook, we obtain the Italian to Greek translation by intentionally triggerring the 504 Gateway Timeout
* We disable retries and run it only once. 
* Logs will not be streamed, so that this notebook can be published on GitHub.

In [ ]:
from scripts.translators import GPTClient
from scripts.data_management import EuroParlManager
from scripts.task import TranslationTask
from scripts.logger import TranslationLogger, logging_config
logging_config(logfile='it-el.log', stream=False) # logs only in .log file, not in stdout
logger = TranslationLogger(logfile='it-el.json')
dm = EuroParlManager()
cli = GPTClient(logger=logger)
cli.client.max_retries = 0 # disable auto-retry from GPTClient

task = TranslationTask(
    target_pairs=[('it', 'el')],
    dm=dm,
    client=cli,
    num_of_sents=400,
    max_retries=0,
    acceptable_range=(360, 480),
    mt_folder='tmp-it-el'
)